In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
util.startLoop()
import stock

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=12)
sx = stock.StockX(ib, 'PLTR')
sx.set_up_frame('1 day', 'ohlcv', start_date="52 weeksAgo", end_date="now")
sx.frames['1 day'].data


Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\PLTR_1_day.csv
Stored data: 265 rows of data
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\PLTR_1_day.csv
Data loaded: 265 rows of data


,open,high,low,close,volume
date,,,,,
2024-01-09,16.51,16.6300,16.28,16.39,317896
2024-01-10,16.49,17.0299,16.40,16.79,367618
2024-01-11,16.81,16.9300,16.17,16.68,347019
2024-01-12,16.68,17.0600,16.62,16.76,304538
2024-01-16,16.59,16.7400,16.29,16.53,275913
...,...,...,...,...,...
2024-12-31,76.75,78.2800,73.65,75.22,492588
2025-01-02,76.19,77.5000,72.42,75.43,632738
2025-01-03,75.57,79.9800,74.63,79.88,543436


In [2]:
import strategies.ta as ta
import strategies.preset_strats as ps
f = sx.frames['1 day']
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=10), [{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], chart_type = 'points')
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=3), [{'color': 'green', 'size': 5}, {'color': 'red', 'size': 5}], chart_type = 'points')
f.add_ta(ta.Ffill(colToFfill='HP_hi_3'), {'dash': 'dot', 'color': 'green', 'width': 1}, chart_type = 'line', row=1)
f.add_ta(ta.Ffill(colToFfill='LP_lo_3'), {'dash': 'dot', 'color': 'red', 'width': 1}, chart_type = 'line', row=1)
ps.require_ta_for_all(f, pointsSpan=10)
ps.ma_ta(f, [21, 50, 200])
f.plot()
f.data.columns

Index(['open', 'high', 'low', 'close', 'volume', 'HP_hi_10', 'LP_lo_10',
       'HP_hi_3', 'LP_lo_3', 'FFILL_HP_hi_3', 'FFILL_LP_lo_3', 'ATR_50',
       'Res_1', 'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper',
       'Res_2_Lower', 'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2',
       'Sup_2_Upper', 'Sup_2_Lower', 'MA_cl_21', 'MA_cl_50', 'MA_cl_200'],
      dtype='object')

# Place orders using raw prices values

In [3]:
import sb_order
ox = sb_order.OrderX('Strat1', ib, 'TSLA', 'LONG')
ox.set_entry(qty=8, outsideRth=True, limitPrice=400.00)
ox.add_bracket_order(qtyPct=25, stop_price=370.00, target_price=420.00)
ox.add_bracket_order(qtyPct=25, stop_price=350.00, target_price=470.00)
ox.add_stop_order(qtyPct=50, stop_price=395.00)
ox.place_orders(delay_between_orders=1)

Stop quotas:  QtyPct: 25, Qty: 2, Quotas: 6
Stop quotas:  QtyPct: 25, Qty: 2, Quotas: 4
Stop quotas:  QtyPct: 50, Qty: 4, Quotas: 0


In [4]:
ox.cancel_orders()

# Place Orders using DataFrame and column names to get prices

In [3]:
import sb_order
import pandas as pd

    
print(f.data['Res_2'].iat[-1])
print(f.data['Res_1'].iat[-1])
print(f.data['Sup_1'].iat[-1])
print(f.data['Sup_2'].iat[-1])
print(f.data['MA_cl_50'].iat[-1])
print(f.data['FFILL_HP_hi_3'].iat[-1])


oxd = sb_order.OrderXData('Strat1', ib, 'TSLA', 'LONG')
oxd.set_entry(f.data, qty=8, outsideRth=True, limitPrice='close')
oxd.add_bracket_order(f.data, qtyPct=25, stop_price='Sup_1', target_price='FFILL_HP_hi_3')
oxd.add_bracket_order(f.data, qtyPct=25, stop_price='Sup_2', target_price='Res_1')
oxd.add_stop_order(f.data, qtyPct=50, stop_price='Sup_1')
oxd.place_orders(f.data, delay_between_orders=0)
print("Connection status:", ib.isConnected())

nan
488.54
362.8
271.0
346.6318
465.33
Stop quotas:  QtyPct: 25, Qty: 2, Quotas: 6
Stop quotas:  QtyPct: 25, Qty: 2, Quotas: 4
Stop quotas:  QtyPct: 50, Qty: 4, Quotas: 0
Connection status: True


In [4]:
oxd.modify_stop(f.data, stopNum=1, stop_price='MA_cl_21')
oxd.modify_stop(f.data, stopNum=2, stop_price='MA_cl_50')

Old price: 362.8, New price: 415.68, new > old = True
Old price: 271.0, New price: 346.63, new > old = True


In [4]:
display(oxd.get_orders_as_df())
display(oxd.get_orders_status_as_df()) # gets auto updated live when connected to IB

,parentId,orderId,clientId,permId,action,totalQuantity,orderType,ocaGroup,ocaType,orderRef,lmtPrice,auxPrice,outsideRth,tif,goodTillDate,transmit
0,0,595,12,551174510,BUY,2.0,LMT,,0,Strat1_Entry_1,403.84,0.0,True,,,False
1,595,596,12,551174511,SELL,2.0,STP,,0,Strat1_Stop_1_pct25,0.00,362.8,False,,,False
2,595,597,12,551174512,SELL,2.0,LMT,,0,Strat1_Tget_1_pct25,465.33,0.0,False,,,True
3,0,598,12,551174513,BUY,2.0,LMT,,0,Strat1_Entry_2,403.84,0.0,True,,,False
4,598,599,12,551174514,SELL,2.0,STP,,0,Strat1_Stop_2_pct25,0.00,271.0,False,,,False
5,598,600,12,551174515,SELL,2.0,LMT,,0,Strat1_Tget_2_pct25,488.54,0.0,False,,,True
6,0,601,12,551174516,BUY,4.0,LMT,,0,Strat1_Entry_3,403.84,0.0,True,,,False
7,601,602,12,551174517,SELL,4.0,STP,,0,Strat1_Stop_3_pct50,0.00,362.8,False,,,True


,orderId,status,filled,remaining,avgFillPrice,permId,parentId,lastFillPrice,clientId,whyHeld,mktCapPrice
0,595,Submitted,0.0,2.0,0.0,551174510,0,0.0,12,,0.0
1,596,PreSubmitted,0.0,2.0,0.0,551174511,595,0.0,12,"child,trigger",0.0
2,597,PreSubmitted,0.0,2.0,0.0,551174512,595,0.0,12,child,0.0
3,598,Submitted,0.0,2.0,0.0,551174513,0,0.0,12,,0.0
4,599,PreSubmitted,0.0,2.0,0.0,551174514,598,0.0,12,"child,trigger",0.0
5,600,PreSubmitted,0.0,2.0,0.0,551174515,598,0.0,12,child,0.0
6,601,Submitted,0.0,4.0,0.0,551174516,0,0.0,12,,0.0
7,602,PreSubmitted,0.0,4.0,0.0,551174517,601,0.0,12,"child,trigger",0.0


In [5]:
oxd.cancel_orders()

# Using OrderXData to update current stock positions

In [8]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
# util.startLoop()
import stock

ib = IB()
# ib.connect('127.0.0.1', 7496, clientId=12)
sx = stock.StockX(ib, 'TSLA')
sx.set_up_frame('1 day', 'ohlcv', start_date="52 weeksAgo", end_date="now")
sx.frames['1 day'].data

Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Stored data: 270 rows of data
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Data loaded: 270 rows of data


,open,high,low,close,volume
date,,,,,
2024-01-09,239.45,240.50,232.04,234.20,815477
2024-01-10,235.51,236.54,231.33,233.72,778458
2024-01-11,235.51,236.00,225.00,225.05,872427
2024-01-12,222.70,225.34,217.15,218.95,1023558
2024-01-16,214.80,223.49,212.18,220.55,949389
...,...,...,...,...,...
2024-12-26,465.16,465.33,451.02,454.13,508124
2024-12-27,449.37,450.00,426.50,431.66,559640
2024-12-30,419.40,427.00,415.75,417.41,411128


In [19]:
import strategies.ta as ta
import strategies.preset_strats as ps
f = sx.frames['1 hour']
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=10), [{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], chart_type = 'points')
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=3), [{'color': 'green', 'size': 5}, {'color': 'red', 'size': 5}], chart_type = 'points')
f.add_ta(ta.Ffill(colToFfill='HP_hi_3'), {'dash': 'dot', 'color': 'green', 'width': 1}, chart_type = 'line', row=1)
f.add_ta(ta.Ffill(colToFfill='LP_lo_3'), {'dash': 'dot', 'color': 'red', 'width': 1}, chart_type = 'line', row=1)
ps.require_ta_for_all(f, pointsSpan=10)
ps.ma_ta(f, [21, 50, 200])
f.plot()
f.data.columns

KeyError: '1 hour'

In [ ]:
# strategies for stops.  stopX
# what to use to swicth stop metrics
# 1. rRatio - risk reward ratio.  if > 2 then swictch to using MA8 as stop
# 2. Accelertaion is > x then switch to using MA8 as stop
# 3. If price is > x% from support to resistance then switch to using MA8 as stop

sm = StopX(ls='LONG', data=f.data, entry='close')   # uses close as entry price to help compute current risk reward ratio as time moves on and prices change. can vlaues also
sm.init_stop(stopCol='Sup_1', offsetPct=1, offsetVal=0.01) # column to use as stop. shift is the number of bars to lag the stop
sm.new_stop(trig='rRatio', trigVal=2, newStop='MA_cl_21', delay=1, offsetPct=1, offsetVal=0.01)  # if rRatio is > 2 then switch to using MA_cl_21 as stop. if trig is rRatio then cumpute rRatio using entry price, init stop price and current price becaseu each rRatio is unique to the current bar and the current trade
sm.new_stop(trig='accel',  trigVal=0.5, newStop='MA_cl_8',  delay=1, offsetPct=1, offsetVal=0.01) # if acceleration is > 0.5 then switch to using MA_cl_8 as stop
sm.new_stop(trig='priceDist', trigVal=0.5, newStop='MA_cl_8',  delay=1, offsetPct=1, offsetVal=0.01) # if price is > 50% from support to resistance then switch to using MA_cl_8 as stop
sm.update_stops(f.data)
sm.get_stop(asValue=False) # returns the column name (if asValue=False) of the stop for the current bar.  gets fed back into the order manager

In [ ]:
import sb_order
import pandas as pd

    
print(f.data['Res_2'].iat[-1])
print(f.data['Res_1'].iat[-1])
print(f.data['Sup_1'].iat[-1])
print(f.data['Sup_2'].iat[-1])
print(f.data['MA_cl_50'].iat[-1])
print(f.data['FFILL_HP_hi_3'].iat[-1])


oxd = sb_order.OrderXData('Strat1', ib, 'TSLA', 'LONG')
oxd.modify_stop(f.data, stopNum=1, stop_price='MA_cl_21')
print("Connection status:", ib.isConnected())

In [5]:
portfolio = ib.portfolio()
portfolio

[PortfolioItem(contract=Stock(conId=76792991, symbol='TSLA', right='0', primaryExchange='ISLAND', currency='USD', localSymbol='TSLA', tradingClass='NMS'), position=8.0, marketPrice=413.0, marketValue=3304.0, averageCost=410.3008, unrealizedPNL=21.59, realizedPNL=0.0, account='DU1780937')]

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

from ib_insync import *
# util.startLoop()
import stock

ib = IB()
# ib.connect('127.0.0.1', 7496, clientId=12)
sx = stock.StockX(ib, 'TSLA')
sx.set_up_frame('1 day', 'ohlcv', start_date="52 weeksAgo", end_date="now")
sx.frames['1 day'].data


Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Stored data: 270 rows of data
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Data loaded: 270 rows of data


,open,high,low,close,volume
date,,,,,
2024-01-09,239.45,240.50,232.04,234.20,815477
2024-01-10,235.51,236.54,231.33,233.72,778458
2024-01-11,235.51,236.00,225.00,225.05,872427
2024-01-12,222.70,225.34,217.15,218.95,1023558
2024-01-16,214.80,223.49,212.18,220.55,949389
...,...,...,...,...,...
2024-12-26,465.16,465.33,451.02,454.13,508124
2024-12-27,449.37,450.00,426.50,431.66,559640
2024-12-30,419.40,427.00,415.75,417.41,411128


In [2]:
import pandas as pd
from typing import Union, List, Dict
from dataclasses import dataclass, field
import math
import numpy as np
from data import historical_data as hd
import strategies.ta as ta
import strategies.preset_strats as ps





import pandas as pd
from typing import Union, List, Dict
from dataclasses import dataclass

class OrderXStatus:
    ENTRY_PRICE_PENDING = 'ENTRY_PRICE_PENDING'
    ENTRY_PRICE_FOUND = 'ENTRY_PRICE_FOUND'
    IN_TRADE = 'IN_TRADE'
    CANCELLED = 'CANCELLED'
    INIT_STOP_PRICE_PENDING = 'INIT_STOP_PRICE_PENDING'
    INIT_STOP_PRICE_FOUND = 'INIT_STOP_PRICE_FOUND'
    TARGET_PRICE_PENDING = 'TARGET_PRICE_PENDING'
    TARGET_PRICE_FOUND = 'TARGET_PRICE_FOUND'
    TRAIL_PRICE_PENDING = 'TRAIL_PRICE_PENDING'
    TRAIL_PRICE_FOUND = 'TRAIL_PRICE_FOUND'
    STOPPED_OUT = 'STOPPED_OUT'
    TARGET_HIT = 'TARGET_HIT'

@dataclass
class BaseClassX:
    name: str = ''
    ls: str = ''
    priceCol: str = ''
    price:float = np.nan
    offsetVal: float = 0.0

    def get_price(self, data:pd.DataFrame, barsAgo:int=1) -> float:
        pass
    
    def reset(self):
        self.price = np.nan

    def has_triggered(self, data, priceType) -> bool:
        if self.ls == 'LONG' and priceType in ['entry', 'target']: return data['high'].iat[-1] >= self.price
        if self.ls == 'SHORT' and priceType in ['entry', 'target']: return data['low'].iat[-1] <= self.price
        if self.ls == 'LONG' and priceType == 'stop': return data['low'].iat[-1] <= self.price
        if self.ls == 'SHORT' and priceType == 'stop': return data['high'].iat[-1] >= self.price

@dataclass
class EntryX(BaseClassX):

    def get_price(self, data:pd.DataFrame, barsAgo:int=1) -> float:
        if self.price > 0:
            return self.price
        breaksPrevHigh = data['close'].iat[-barsAgo] > data['high'].iat[-barsAgo-1]
        if breaksPrevHigh:
            self.price = data['close'].iat[-barsAgo]
        return self.price


@dataclass
class StopX(BaseClassX):

    def get_price(self, data:pd.DataFrame, barsAgo:int=1) -> float:
        if self.price > 0: return self.price
        self.price = data[self.priceCol].iat[-barsAgo-1]
        return self.price


@dataclass
class TargetX(BaseClassX):
    name: str = ''
    ls: str = ''
    price:float = np.nan

    def get_price(self, data:pd.DataFrame, barsAgo:int=1) -> float:
        if self.price > 0:
            return self.price
        self.price = data['close'].iat[-barsAgo]
        return self.price
    

@dataclass
class TrailX(BaseClassX):
    name: str = ''
    ls: str = ''
    price:float = np.nan
    initType: str = 'rrr'
    initTrigVal: float = 0.0
    priceCol: str = ''


    def get_price(self, data:pd.DataFrame, barsAgo:int=1) -> float:
        self.price = data[self.priceCol].iat[-barsAgo-1]
        return self.price
    



@dataclass
class RiskX:
    name: str = ''
    ls: str = ''
    risk: float = 0.0
    reward: float = 0.0
    rRatio: float = 0.0

    def get_value(self, entryPrice, stopPrice, priceNow):  
        if self.ls == 'LONG':
            self.risk = entryPrice - stopPrice
            self.reward = priceNow - entryPrice
        elif self.ls == 'SHORT':
            self.risk = stopPrice - entryPrice
            self.reward = entryPrice - priceNow
        if self.risk == 0: return 0
        self.rRatio = self.reward / self.risk
        return self.rRatio

    
    def reset(self):
        self.risk = 0.0
        self.reward = 0.0
        self.rRatio = 0.0


@dataclass
class TraceX:
    name: str = ''
    ls: str = ''
    price:float = np.nan

    def get_price(self, data:pd.DataFrame, barsAgo:int=1) -> float:
        if self.price > 0:
            return self.price
        self.price = data['close'].iat[-barsAgo]
        return self.price
    
    def reset(self):
        self.price = np.nan

@dataclass
class AccelX:
    name: str = ''
    ls: str = ''
    price:float = np.nan

    def get_price(self, data:pd.DataFrame, barsAgo:int=1) -> float:
        if self.price > 0:
            return self.price
        self.price = data['close'].iat[-barsAgo]
        return self.price
    
    def reset(self):
        self.price = np.nan


@dataclass
class OrderPriceX:
    name: str = ''
    ls: str = ''
    includeTarget: bool = False
    entry: EntryX = None
    stop: StopX = None
    target: TargetX = None
    trails: list = field(default_factory=list) # list of TrailX objects
    risk: RiskX = None
    trace: TraceX = None
    accel: AccelX = None
    stopPrice: float = np.nan
    stopName: str = ''
    offsetVal: float = 0.0
    activeTrails: list = field(default_factory=list) # list of TrailX objects
    entryIndex: pd.DatetimeIndex = None

    def __post_init__(self):
        self.status = OrderXStatus.ENTRY_PRICE_PENDING
        self.trails.reverse()
        self.activeTrails = []

    def price_found(self, price):
        return not math.isnan(price)


    def set_columns(self, df:pd.DataFrame):
        df[self.entry.name] = np.nan
        df[self.stop.name] = np.nan
        df[self.target.name] = np.nan
        df[[t.name for t in self.trails]] = np.nan
        df[self.risk.name] = np.nan
        df[self.trace.name] = np.nan
        df[self.accel.name] = np.nan
        df['StopName'] = ''

    def reset(self):
        self.status = OrderXStatus.ENTRY_PRICE_PENDING
        self.entry.reset()
        self.stop.reset()
        self.target.reset()
        for t in self.trails: t.reset()
        self.risk.reset()
        self.trace.reset()
        self.stopPrice = np.nan
        self.stopName = ''
        self.activeTrails = []

    def get_init_value(self, df:pd.DataFrame, initType:str):
        if initType == 'rrr':
            # print(f"get_init_value = 'rrr' {self.entry.price=}, {self.stopPrice=}, {df['close'].iat[-1]=}")
            return self.risk.get_value(self.entry.price, self.stop.price, df['close'].iat[-1])
        
    def compute_stop(self, df:pd.DataFrame, newStopPrice:float):
        limited_price = df['low'].iat[-1] -0.01
        max_stop = min(newStopPrice, limited_price)
        if max_stop > self.stopPrice:
            self.stopPrice = max_stop
            self.stopName = self.stop.name
        return self.stopPrice

    def run_row(self, df:pd.DataFrame):
        # check if reset required 
        if self.status in [OrderXStatus.TARGET_HIT, OrderXStatus.STOPPED_OUT]:
            self.reset()
            self.status = OrderXStatus.ENTRY_PRICE_PENDING

        # find entry price
        if self.status == OrderXStatus.ENTRY_PRICE_PENDING:
            if self.entry.get_price(df) > 0:
                self.status = OrderXStatus.ENTRY_PRICE_FOUND
                self.entryIndex = df.index[-1]

        # set init stop price
        if self.status == OrderXStatus.ENTRY_PRICE_FOUND:
            self.stopPrice =  self.stop.get_price(df, barsAgo=1) # stops are always set to main stop price
            if self.stopPrice > 0:
                self.stopName = self.stop.name
                self.status = OrderXStatus.INIT_STOP_PRICE_FOUND

        # set target price if required
        if self.includeTarget:
            if self.status == OrderXStatus.ENTRY_PRICE_FOUND:
                if self.target.get_price(df) > 0:
                    self.status = OrderXStatus.TARGET_PRICE_FOUND

        # skip entry index is now 
        if self.status in [OrderXStatus.ENTRY_PRICE_FOUND, OrderXStatus.INIT_STOP_PRICE_FOUND, OrderXStatus.TARGET_PRICE_FOUND]:
            if self.entryIndex == df.index[-1]:
                return
        
        # see if entry price has been hit
        if self.status in [OrderXStatus.INIT_STOP_PRICE_FOUND, OrderXStatus.TARGET_PRICE_FOUND]:
            if self.entry.has_triggered(df, priceType='entry'):
                self.status = OrderXStatus.IN_TRADE
            if self.stop.has_triggered(df, priceType='stop'):
                self.status = OrderXStatus.STOPPED_OUT

        # see if stopped out
        if self.status == OrderXStatus.IN_TRADE:
            if self.stop.has_triggered(df, priceType='stop'):
                self.status = OrderXStatus.STOPPED_OUT
            for t in self.trails:
                if t.has_triggered(df, priceType='stop'):
                    self.status = OrderXStatus.STOPPED_OUT

        # see if target price has been hit
        if self.status == OrderXStatus.IN_TRADE:
            if self.target.has_triggered(df, priceType='target'):
                self.status = OrderXStatus.TARGET_HIT
        
        if self.status == OrderXStatus.IN_TRADE:
            self.activeTrails += [t for t in self.trails if self.get_init_value(df, t.initType) > t.initTrigVal and t not in self.activeTrails]
            if len(self.activeTrails) > 0:
                active_trails = [(t.name, t.get_price(df)) for t in self.activeTrails]
                # print(f"{df.index[-1]} activeTrails: {active_trails}")
                
                # Find the trail with the maximum stop price
                max_trail = max(active_trails, key=lambda x: x[1])
                if max_trail[1] > self.stopPrice:
                    limit_price = df['low'].iat[-1] -0.01
                    self.stopName = max_trail[0]
                    self.stopPrice = max(min(max_trail[1], limit_price), self.stopPrice)
                

            
   

        
# stopx.get_stop(asValue=True)
f = sx.frames['1 day']
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=10), [{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], chart_type = 'points')
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=3), [{'color': 'green', 'size': 5}, {'color': 'red', 'size': 5}], chart_type = 'points')
f.add_ta(ta.Ffill(colToFfill='HP_hi_3'), {'dash': 'dot', 'color': 'green', 'width': 1}, chart_type = 'line', row=1)
f.add_ta(ta.Ffill(colToFfill='LP_lo_3'), {'dash': 'dot', 'color': 'red', 'width': 1}, chart_type = 'line', row=1)
f.add_ta(ta.LowestHighest(hi_col='high', lo_col='low', span=3), [{'color': 'green', 'size': 1}, {'color': 'red', 'size': 5}], chart_type = 'points')
f.add_ta(ta.LowestHighest(hi_col='high', lo_col='low', span=1), [{'color': 'green', 'size': 1}, {'color': 'red', 'size': 5}], chart_type = 'points')
ps.require_ta_for_all(f, pointsSpan=10)
ps.ma_ta(f, [8, 21, 50, 200])
f.update_data(f.data)


opx  = OrderPriceX(name='Strat1', ls='LONG', includeTarget=False)
opx.entry = EntryX(name='Entry1', ls='LONG')
opx.stop = StopX(name='initStop', ls='LONG', priceCol='LoIst_lo_3', offsetVal=0.50)
opx.target = TargetX(name='Target1', ls='LONG')
opx.trails += [
    TrailX(name='Stop1', ls='LONG', initType='rrr', initTrigVal=1, priceCol='MA_cl_50', offsetVal=0.01),
    TrailX(name='Stop2', ls='LONG', initType='rrr', initTrigVal=2, priceCol='FFILL_LP_lo_3', offsetVal=0.01),
    TrailX(name='Stop3', ls='LONG', initType='rrr', initTrigVal=3, priceCol='MA_cl_21', offsetVal=0.01),
    TrailX(name='Stop4', ls='LONG', initType='rrr', initTrigVal=4, priceCol='MA_cl_8', offsetVal=0.01),
    TrailX(name='Stop5', ls='LONG', initType='rrr', initTrigVal=5, priceCol='LoIst_lo_1', offsetVal=0.01),
    ]
opx.risk = RiskX(name='Risk1', ls='LONG')
opx.trace = TraceX(name='Trace1', ls='LONG')
opx.accel = AccelX(name='Accel1', ls='LONG')

opx.set_columns(f.data)
print(opx.status)
print(f.data.columns)


for index, row in f.data.iterrows():
    dfx = f.data.loc[:index].copy()
    if len (dfx) < 2:
        continue
    opx.run_row(dfx)
    # print(f"{index} Ent:{opx.entry.price}, StpName: {opx.stopName}, Stp:{opx.stopPrice} {opx.status}")
    f.data.loc[index, opx.entry.name] = opx.entry.price
    f.data.loc[index, 'StopPrice'] = opx.stopPrice 
    f.data.loc[index, 'StopName'] = opx.stopName
    



f.add_ta(ta.AddColumn('Entry1'), [{'color': 'yellow', 'size': 3}, {'color': 'red', 'size': 3}], chart_type='points', row=1)
f.add_ta(ta.AddColumn('StopPrice'), [{'color': 'magenta', 'size': 5}], chart_type='points', row=1, nameCol='StopName')
f.add_ta(ta.AddColumn('Risk1'), {'dash': 'solid', 'color': 'red', 'width': 3}, chart_type='lines+markers', row=3)
    
# f.chart.refesh(f.data)
print(opx.status)
display(f.data[['close', 'Entry1', 'StopPrice', 'StopName']].tail(50))
f.plot(height=1200)


ENTRY_PRICE_PENDING
Index(['open', 'high', 'low', 'close', 'volume', 'HP_hi_10', 'LP_lo_10',
       'HP_hi_3', 'LP_lo_3', 'FFILL_HP_hi_3', 'FFILL_LP_lo_3', 'HiIst_hi_3',
       'LoIst_lo_3', 'HiIst_hi_1', 'LoIst_lo_1', 'ATR_50', 'Res_1',
       'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper', 'Res_2_Lower',
       'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2', 'Sup_2_Upper',
       'Sup_2_Lower', 'MA_cl_8', 'MA_cl_21', 'MA_cl_50', 'MA_cl_200', 'Entry1',
       'initStop', 'Target1', 'Stop1', 'Stop2', 'Stop3', 'Stop4', 'Stop5',
       'Risk1', 'Trace1', 'Accel1', 'StopName'],
      dtype='object')
ENTRY_PRICE_PENDING


,close,Entry1,StopPrice,StopName
date,,,,
2024-10-22,217.50,NaN,NaN,
2024-10-23,239.50,239.5,215.260000,initStop
2024-10-24,257.43,239.5,215.260000,initStop
2024-10-25,267.09,239.5,229.831400,Stop1
2024-10-28,260.10,239.5,230.873800,Stop1
2024-10-29,259.80,239.5,231.749400,Stop1
2024-10-30,256.08,239.5,232.505400,Stop1
2024-10-31,250.85,239.5,233.238800,Stop1
2024-11-01,249.37,239.5,233.783800,Stop1


In [7]:
f.data['StopName'].to_list()

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 '',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 '',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 '',
 '',
 '',
 '',
 '',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'Stop1',
 'Stop1',
 'Stop1',
 'Stop1',
 'Stop1',
 'Stop1',
 'Stop1',
 'Stop1',
 '',
 '',
 '',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'initStop',
 'i

In [4]:
# f.data[['signal', 'risk', 'rRatio', 'StopX', 'Col_StopX']].tail(50)
f.data.columns

Index(['open', 'high', 'low', 'close', 'volume', 'HP_hi_10', 'LP_lo_10',
       'Entry1', 'initStop', 'Target1', 'Stop1', 'Stop2', 'Stop3', 'Stop4',
       'Stop5', 'Risk1', 'Trace1', 'Accel1', 'StopName', 'StopPrice'],
      dtype='object')

In [ ]:
# changed delay to shift in new_stop method. this means the number of bars to shift to get the value. 0 means current bar, 1 means previous bar, etc
# Risk should be calcualted upon init of the stop manager and stored as a set value.  The rrRatio is computed on the fly as the price changes and uses the original risk value compared to the current reward value (current price - entry price)
# we need to add each new_stop type as a separet method that is called when the trigger is met.  This method should set the new stop value and offset values. the reason the ways of getting trig values is too varied to have a single method.
# eg new_stop_rRatio, new_stop_accel, new_stop_priceDist 
# remove all the doc strings for now whilst debugging and testing the code


In [18]:
import pandas as pd
import numpy as np
from dataclasses import dataclass

def generate_signal(df, n):
    lower_high = df['high'] < df['high'].shift(n)
    breaks_prev_bar = df['close'] > df['low'].shift(1)

    df['signal'] = np.nan
    df['signal'] = np.where(lower_high & breaks_prev_bar, True, np.nan)

    return df


@dataclass
class RiskRewardRatio:
    entry: float = 0
    stop: float = 0
    price: float = 0
    risk: float = 0
    reward: float = 0
    rrRatio: float = 0

    def get_ratio(self):
        self.risk = self.entry - self.stop
        self.reward = self.price - self.entry
        self.rrRatio = round(self.reward / self.risk, 1)
        print(f"Entry: {self.entry}, Stop: {self.stop}, Price: {self.price}, Risk: {self.risk}, Reward: {self.reward}, RR Ratio: {self.rrRatio}")
        return self.rrRatio
    
def generate_risk_at_points(df):
    df['risk'] = np.where(df['LP_lo_10'] > 0, df['high'] - df['low'], np.nan)   
    df['rRatio'] = np.nan





    for i, row in df.iterrows():
        dfx = df.loc[:i]
        if len(dfx) < 2:
            continue
        if dfx['signal'].iat[-1] > 0: 
            rrr = RiskRewardRatio()
            rrr.entry = dfx['close'].iat[-2]
            rrr.stop = dfx['LP_lo_10'].iat[-2]
            rrr.price = dfx['close'].iat[-1]
            dfx.at[i, 'rRatio'] = rrr.get_ratio()
    return dfx

f.data = generate_signal(f.data, 3)
f.data = generate_risk_at_points(f.data)
# df.loc[df['signal'] == True][['risk']].dropna()
f.data[['signal', 'risk', 'rRatio', 'StopX']].dropna(how='all').tail(50)

Entry: 16.53, Stop: nan, Price: 16.39, Risk: nan, Reward: -0.14000000000000057, RR Ratio: nan
Entry: 16.39, Stop: 16.05, Price: 16.4, Risk: 0.33999999999999986, Reward: 0.00999999999999801, RR Ratio: 0.0
Entry: 16.46, Stop: nan, Price: 16.35, Risk: nan, Reward: -0.10999999999999943, RR Ratio: nan
Entry: 16.35, Stop: nan, Price: 17.09, Risk: nan, Reward: 0.7399999999999984, RR Ratio: nan
Entry: 16.09, Stop: nan, Price: 16.33, Risk: nan, Reward: 0.23999999999999844, RR Ratio: nan
Entry: 16.33, Stop: 16.03, Price: 17.02, Risk: 0.29999999999999716, Reward: 0.6900000000000013, RR Ratio: 2.3
Entry: 25.05, Stop: nan, Price: 24.01, Risk: nan, Reward: -1.0399999999999991, RR Ratio: nan
Entry: 25.19, Stop: nan, Price: 25.41, Risk: nan, Reward: 0.21999999999999886, RR Ratio: nan
Entry: 23.4, Stop: nan, Price: 22.74, Risk: nan, Reward: -0.6600000000000001, RR Ratio: nan
Entry: 22.74, Stop: nan, Price: 23.59, Risk: nan, Reward: 0.8500000000000014, RR Ratio: nan
Entry: 24.04, Stop: nan, Price: 23.81

,signal,risk,rRatio,StopX
date,,,,
2024-09-27,1.0,NaN,NaN,NaN
2024-10-16,1.0,NaN,NaN,NaN
2024-10-17,1.0,NaN,NaN,NaN
2024-10-18,1.0,NaN,NaN,NaN
2024-10-31,NaN,2.42,NaN,40.9
2024-11-01,1.0,NaN,0.5,40.9
2024-11-04,NaN,NaN,NaN,40.9
2024-11-05,NaN,NaN,NaN,40.9
2024-11-06,NaN,NaN,NaN,40.9


In [18]:



f.add_ta(ta.AddColumn('Stop1'), {'dash': 'solid', 'color': 'yellow', 'width': 3}, chart_type = 'line', row=1)
print(f.data.columns)
f.plot()

Index(['open', 'high', 'low', 'close', 'volume', 'HP_hi_10', 'LP_lo_10',
       'HP_hi_3', 'LP_lo_3', 'FFILL_HP_hi_3', 'FFILL_LP_lo_3', 'ATR_50',
       'Res_1', 'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper',
       'Res_2_Lower', 'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2',
       'Sup_2_Upper', 'Sup_2_Lower', 'MA_cl_21', 'MA_cl_50', 'MA_cl_200',
       'risk', 'rRatio', 'Stop1', 'Col_Stop1'],
      dtype='object')
